# Đồ án cuối kỳ

Họ tên: Võ Nhật Thịnh

MSSV: 1712796


# 1. nguồn dữ liệu

>Khái quát về dữ liệu:

Bộ dữ liệu cung cấp thông tin tình hình dịch COVID-19 đang diễn ra trên thế giới, mức độ lây lan của chúng, tổng số ca mắc, ca qua khỏi và ca tử vong theo thời gian thực, tức bộ dữ liệu luôn được cập nhật từng ngày từng giờ giúp ta có 1 cái nhìn trực quan nhất về dịch

Nguồn dữ liệu: https://www.kaggle.com/sudalairajkumar/novel-corona-virus-2019-dataset

>license của chủ đề:

Bộ dữ liệu có bản quyền thuộc về Creative Commons Attribution 4.0 International (CC BY 4.0) của trường Johns Hopkins University
để sử dụng bộ dữ liệu cho mục đích nghiên cứu, ta trích dẫn nguồn dữ liệu:

"Dong E, Du H, Gardner L. An interactive web-based dashboard to track COVID-19 in real time. Lancet Inf Dis. 20(5):533-534. doi: 10.1016/S1473-3099(20)30120-1"

chi tiết tham khảo: https://github.com/CSSEGISandData/COVID-19/blob/master/README.md

> cách dữ liệu được thu thập:

nguồn dữ liệu được thu thập từ 3 nguồn chính: 

    data tổng hợp từ các tổ chức lớn trên thế giới (WHO, ECDC, DXY,..)
    
    data từ các bang của US
    
    data từ các nước/ vùng khác ngoài US

# 2. khám phá dữ liệu

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from matplotlib import pyplot
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error

Các bước xử lý dữ liệu:

đọc qua dữ liệu thì thấy có 2 loại dữ liệu:

các file time series có các cột là các ngày, giá trị tương ứng của các ô trong ngày đó là các ca mắc bệnh và các dòng là các nước

các file covid 19 thì là chi tiết ngày giờ phát hiện ca mắc và các thông tin chi tiết có liên quan

file Summary trước làm để tổng hợp lại các cột trong file

Ý tưởng:

đây là các dữ liệu time series nên dựa theo hướng này để giải quyết

Đọc file TimeSeries.ipynb có ý tưởng độ lại tập file time series: với mỗi dòng trong file đều là các nước, các cột thể hiện giá trị số ca mắc trong ngày đó tương ứng với tên cột vậy ta ta tạo biến kiểu DataFrame hay dictionary để lưu lại vd: Case['England']['29-7-2020']

ý tưởng là vậy trước mắt ta phải hoàn thiện công đoạn xử lý tiền dữ liệu:

giá trị các ô trong file (đặc biệt là các ngày) chưa theo 1 mẫu hoàn chỉnh, cần xử lý về 1 format để tiện xử lý
đọc hiểu các file covid-19

covid_19_data.csv: rất cơ bản, ngày phát hiện ca nhiễm tại quốc gia nào và tình trạng ra sao

COVID19_line_list_data.csv (xóa cột D thừa): chi tiết nên phức tạp hơn file trên, thể hiện theo ngày diễn biến, mỗi dòng là 1 báo cáo tình hình dịch tại nước sở tại (vd: filter cột country theo South Korea), có khoảng ngày tiếp xúc, giới tính, chi tiết triệu chứng, ngày xuất hiện triệu chứng, có đến vũ hán hay không hoặc từ vũ hán đi ra ngoài, ngày đi khám tại bệnh viện và nguồn

COVID19_open_line_list.csv: chi tiết vcl hơn cả file trên, có tất cả cái trên và có cả: tọa độ (chưa biết về cái gì), mấy ông admin xử lý dữ liệu nào (chắc k liên quan), ngày có bệnh (ngày phát, ngày đến khám, ngày xác nhận nhiễm), lịch sử đi lại (ngày, địa điểm), các bệnh lý kèm theo

---------------

11/08: lưu các file vào biến trước r tính sau

# Ở đây ta sử dụng thư viện plotly để vẽ các biểu đồ, nếu chưa cài đặt thì ta sử dụng câu lệnh: 
'pip install plotly'

In [11]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from matplotlib import pyplot
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error

In [12]:
#nạp các file vào biến
covid_19_data = pd.read_csv('covid_19_data.csv')
COVID19_line_list_data = pd.read_csv('COVID19_line_list_data.csv')
COVID19_open_line_list = pd.read_csv('COVID19_open_line_list.csv')

time_series_covid_19_confirmed = pd.read_csv('time_series_covid_19_confirmed.csv')
time_series_covid_19_confirmed_US = pd.read_csv('time_series_covid_19_confirmed_US.csv')
time_series_covid_19_deaths = pd.read_csv('time_series_covid_19_deaths.csv')
time_series_covid_19_deaths_US = pd.read_csv('time_series_covid_19_deaths_US.csv')
time_series_covid_19_recovered = pd.read_csv('time_series_covid_19_recovered.csv')

xem xét xem các dữ liệu có bị missing value hay NULL không

In [13]:
# file covid_19_data.csv:
f = covid_19_data
miss_f = [(c, f[c].isna().mean()*100) for c in f]
miss_f = pd.DataFrame(miss_f, columns=["column_name", "percentage"])
miss_f

,column_name,percentage
0,SNo,0.00000
1,ObservationDate,0.00000
2,Province/State,34.14228
3,Country/Region,0.00000
4,Last Update,0.00000
5,Confirmed,0.00000
6,Deaths,0.00000
7,Recovered,0.00000


xem xét xem các dữ liệu có bị missing value hay NULL không

In [14]:
# file covid_19_data.csv:
f = covid_19_data
miss_f = [(c, f[c].isna().mean()*100) for c in f]
miss_f = pd.DataFrame(miss_f, columns=["column_name", "percentage"])
miss_f

,column_name,percentage
0,SNo,0.00000
1,ObservationDate,0.00000
2,Province/State,34.14228
3,Country/Region,0.00000
4,Last Update,0.00000
5,Confirmed,0.00000
6,Deaths,0.00000
7,Recovered,0.00000


In [15]:
f = COVID19_line_list_data
miss_f = [(c, f[c].isna().mean()*100) for c in f]
miss_f = pd.DataFrame(miss_f, columns=["column_name", "percentage"])
miss_f

,column_name,percentage
0,id,0.000000
1,case_in_country,18.156682
2,reporting date,0.092166
3,Unnamed: 3,100.000000
4,summary,0.460829
5,location,0.000000
6,country,0.000000
7,gender,16.866359
8,age,22.304147
9,symptom_onset,48.110599


In [16]:
f = COVID19_open_line_list
miss_f = [(c, f[c].isna().mean()*100) for c in f]
miss_f = pd.DataFrame(miss_f, columns=["column_name", "percentage"])
miss_f

,column_name,percentage
0,ID,6.746425
1,age,90.450234
2,sex,91.051961
3,city,27.835198
4,province,8.636557
5,country,6.923404
6,wuhan(0)_not_wuhan(1),6.767662
7,latitude,6.930483
8,longitude,6.930483
9,geo_resolution,6.930483


In [17]:
f = time_series_covid_19_confirmed
miss_f = [(c, f[c].isna().mean()*100) for c in f]
miss_f = pd.DataFrame(miss_f, columns=["column_name", "percentage"])
miss_f

,column_name,percentage
0,Province/State,69.548872
1,Country/Region,0.000000
2,Lat,0.000000
3,Long,0.000000
4,1/22/20,0.000000
...,...,...
192,7/28/20,0.000000
193,7/29/20,0.000000
194,7/30/20,0.000000
195,7/31/20,0.000000


In [18]:
f = time_series_covid_19_confirmed_US
miss_f = [(c, f[c].isna().mean()*100) for c in f]
miss_f = pd.DataFrame(miss_f, columns=["column_name", "percentage"])
miss_f

,column_name,percentage
0,UID,0.000000
1,iso2,0.000000
2,iso3,0.000000
3,code3,0.000000
4,FIPS,0.299401
...,...,...
199,7/28/20,0.000000
200,7/29/20,0.000000
201,7/30/20,0.000000
202,7/31/20,0.000000


In [19]:
f = time_series_covid_19_deaths
miss_f = [(c, f[c].isna().mean()*100) for c in f]
miss_f = pd.DataFrame(miss_f, columns=["column_name", "percentage"])
miss_f

,column_name,percentage
0,Province/State,69.548872
1,Country/Region,0.000000
2,Lat,0.000000
3,Long,0.000000
4,1/22/20,0.000000
...,...,...
192,7/28/20,0.000000
193,7/29/20,0.000000
194,7/30/20,0.000000
195,7/31/20,0.000000


In [20]:
f = time_series_covid_19_deaths_US
miss_f = [(c, f[c].isna().mean()*100) for c in f]
miss_f = pd.DataFrame(miss_f, columns=["column_name", "percentage"])
miss_f

,column_name,percentage
0,UID,0.000000
1,iso2,0.000000
2,iso3,0.000000
3,code3,0.000000
4,FIPS,0.299401
...,...,...
200,7/28/20,0.000000
201,7/29/20,0.000000
202,7/30/20,0.000000
203,7/31/20,0.000000


In [21]:
f = time_series_covid_19_recovered
miss_f = [(c, f[c].isna().mean()*100) for c in f]
miss_f = pd.DataFrame(miss_f, columns=["column_name", "percentage"])
miss_f

,column_name,percentage
0,Province/State,73.517787
1,Country/Region,0.000000
2,Lat,0.000000
3,Long,0.000000
4,1/22/20,0.000000
...,...,...
192,7/28/20,0.000000
193,7/29/20,0.000000
194,7/30/20,0.000000
195,7/31/20,0.000000


In [22]:
# tỉ lệ missing value giữa các file chênh lệch với nhau khá nhiều, 
# để thuận tiện cho việc xử lý dữ liệu, ta cần điền giá trị NULL cho file
# và xóa các cột trống dữ liệu:

# file covid_19_data
covid_19_data["Province/State"]= covid_19_data["Province/State"].fillna('Unknown')

# file COVID19_line_list_data
COVID19_line_list_data["case_in_country"]= COVID19_line_list_data["case_in_country"].fillna('Unknown')
COVID19_line_list_data["reporting date"]= COVID19_line_list_data["reporting date"].fillna('Unknown')
COVID19_line_list_data.drop("Unnamed: 3", axis=1)
COVID19_line_list_data["summary"]= COVID19_line_list_data["summary"].fillna('Unknown')
COVID19_line_list_data["gender"]= COVID19_line_list_data["gender"].fillna('Unknown')
COVID19_line_list_data["age"]= COVID19_line_list_data["age"].fillna('Unknown')
COVID19_line_list_data["symptom_onset"]= COVID19_line_list_data["symptom_onset"].fillna('Unknown')
COVID19_line_list_data["If_onset_approximated"]= COVID19_line_list_data["If_onset_approximated"].fillna('Unknown')
COVID19_line_list_data["hosp_visit_date"]= COVID19_line_list_data["hosp_visit_date"].fillna('Unknown')
COVID19_line_list_data["exposure_start"]= COVID19_line_list_data["exposure_start"].fillna('Unknown')
COVID19_line_list_data["exposure_end"]= COVID19_line_list_data["exposure_end"].fillna('Unknown')
COVID19_line_list_data["from Wuhan"]= COVID19_line_list_data["from Wuhan"].fillna('Unknown')
COVID19_line_list_data["symptom"]= COVID19_line_list_data["symptom"].fillna('Unknown')
COVID19_line_list_data.drop("Unnamed: 21", axis=1)
COVID19_line_list_data.drop("Unnamed: 22", axis=1)
COVID19_line_list_data.drop("Unnamed: 23", axis=1)
COVID19_line_list_data.drop("Unnamed: 24", axis=1)
COVID19_line_list_data.drop("Unnamed: 25", axis=1)
COVID19_line_list_data.drop("Unnamed: 26", axis=1)

# file COVID19_open_line_list
COVID19_open_line_list["ID"]= COVID19_open_line_list["ID"].fillna('Unknown')
COVID19_open_line_list["age"]= COVID19_open_line_list["age"].fillna('Unknown')
COVID19_open_line_list["sex"]= COVID19_open_line_list["sex"].fillna('Unknown')
COVID19_open_line_list["city"]= COVID19_open_line_list["city"].fillna('Unknown')
COVID19_open_line_list["province"]= COVID19_open_line_list["province"].fillna('Unknown')
COVID19_open_line_list["country"]= COVID19_open_line_list["country"].fillna('Unknown')
COVID19_open_line_list["wuhan(0)_not_wuhan(1)"]= COVID19_open_line_list["wuhan(0)_not_wuhan(1)"].fillna('Unknown')
COVID19_open_line_list["latitude"]= COVID19_open_line_list["latitude"].fillna('Unknown')
COVID19_open_line_list["longitude"]= COVID19_open_line_list["longitude"].fillna('Unknown')
COVID19_open_line_list["geo_resolution"]= COVID19_open_line_list["geo_resolution"].fillna('Unknown')
COVID19_open_line_list["date_onset_symptoms"]= COVID19_open_line_list["date_onset_symptoms"].fillna('Unknown')
COVID19_open_line_list["date_admission_hospital"]= COVID19_open_line_list["date_admission_hospital"].fillna('Unknown')
COVID19_open_line_list["date_confirmation"]= COVID19_open_line_list["date_confirmation"].fillna('Unknown')
COVID19_open_line_list["symptoms"]= COVID19_open_line_list["symptoms"].fillna('Unknown')
COVID19_open_line_list["lives_in_Wuhan"]= COVID19_open_line_list["lives_in_Wuhan"].fillna('Unknown')
COVID19_open_line_list["travel_history_dates"]= COVID19_open_line_list["travel_history_dates"].fillna('Unknown')
COVID19_open_line_list["travel_history_location"]= COVID19_open_line_list["travel_history_location"].fillna('Unknown')
COVID19_open_line_list["reported_market_exposure"]= COVID19_open_line_list["reported_market_exposure"].fillna('Unknown')
COVID19_open_line_list["additional_information"]= COVID19_open_line_list["additional_information"].fillna('Unknown')
COVID19_open_line_list["chronic_disease_binary"]= COVID19_open_line_list["chronic_disease_binary"].fillna('Unknown')
COVID19_open_line_list["chronic_disease"]= COVID19_open_line_list["chronic_disease"].fillna('Unknown')
COVID19_open_line_list["source"]= COVID19_open_line_list["source"].fillna('Unknown')
COVID19_open_line_list["sequence_available"]= COVID19_open_line_list["sequence_available"].fillna('Unknown')
COVID19_open_line_list["outcome"]= COVID19_open_line_list["outcome"].fillna('Unknown')
COVID19_open_line_list["date_death_or_discharge"]= COVID19_open_line_list["date_death_or_discharge"].fillna('Unknown')
COVID19_open_line_list["notes_for_discussion"]= COVID19_open_line_list["notes_for_discussion"].fillna('Unknown')
COVID19_open_line_list["location"]= COVID19_open_line_list["location"].fillna('Unknown')
COVID19_open_line_list["admin3"]= COVID19_open_line_list["admin3"].fillna('Unknown')
COVID19_open_line_list["admin2"]= COVID19_open_line_list["admin2"].fillna('Unknown')
COVID19_open_line_list["admin1"]= COVID19_open_line_list["admin1"].fillna('Unknown')
COVID19_open_line_list["country_new"]= COVID19_open_line_list["country_new"].fillna('Unknown')
COVID19_open_line_list["admin_id"]= COVID19_open_line_list["admin_id"].fillna('Unknown')
COVID19_open_line_list["data_moderator_initials"]= COVID19_open_line_list["data_moderator_initials"].fillna('Unknown')
COVID19_open_line_list.drop("Unnamed: 33", axis=1)
COVID19_open_line_list.drop("Unnamed: 34", axis=1)
COVID19_open_line_list.drop("Unnamed: 35", axis=1)
COVID19_open_line_list.drop("Unnamed: 36", axis=1)
COVID19_open_line_list.drop("Unnamed: 37", axis=1)
COVID19_open_line_list.drop("Unnamed: 38", axis=1)
COVID19_open_line_list.drop("Unnamed: 39", axis=1)
COVID19_open_line_list.drop("Unnamed: 40", axis=1)
COVID19_open_line_list.drop("Unnamed: 41", axis=1)
COVID19_open_line_list.drop("Unnamed: 42", axis=1)
COVID19_open_line_list.drop("Unnamed: 43", axis=1)
COVID19_open_line_list.drop("Unnamed: 44", axis=1)

# file time_series_covid_19_confirmed
time_series_covid_19_confirmed["Province/State"]= time_series_covid_19_confirmed["Province/State"].fillna('Unknown')

# file COVID19_line_list_data
time_series_covid_19_confirmed_US["FIPS"]= time_series_covid_19_confirmed_US["FIPS"].fillna('Unknown')

# file time_series_covid_19_deaths
time_series_covid_19_deaths["Province/State"]= time_series_covid_19_deaths["Province/State"].fillna('Unknown')

# file time_series_covid_19_deaths_US
time_series_covid_19_deaths_US["FIPS"]= time_series_covid_19_deaths_US["FIPS"].fillna('Unknown')

# file time_series_covid_19_recovered
time_series_covid_19_recovered["Province/State"]= time_series_covid_19_recovered["Province/State"].fillna('Unknown')

Sau khi xử lý data, ta thấy dữ liệu các file COVID19_line_list_data.csv và COVID19_open_line_list.csv có nhiều giá trị NULL, điều này sẽ gây khó khăn trong việc trực quan và tìm insights của dữ liệu nên ở đây ta chỉ xét file covid_19_data.csv

In [23]:
# đảo ngược thời gian trong file để tìm ca mới nhất
Data = covid_19_data[covid_19_data['ObservationDate'] == max(covid_19_data['ObservationDate'])].reset_index()

In [24]:
# tính các ca hiện tại đang mắc trên thế giới
covid_19_data['Activated'] = covid_19_data['Confirmed'] - covid_19_data['Deaths'] - covid_19_data['Recovered']